# Test set evaluation of Gender image classification models

## UTKFace dataset

In [1]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix

In [8]:
# UTKFace image directories (unsorted and sorted)
TEST_DIR = 'UTKFace/'
TEST_SORT_DIR = 'UTKFace sorted/'

In [4]:
# labels - age, gender, ethnicity
image_paths = []
age_labels = []
gender_labels = []

for filename in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, filename)
    temp = filename.split('_')
    age = int(temp[0])
    gender = int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

In [7]:
# Move UTKFace images into unique directories based on their gender label
# Need only to run once to organize
m_dir = 'UTKFace sorted/male'
f_dir = 'UTKFace sorted/female'

import shutil

for filename in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, filename)
    temp = filename.split('_')
    gender = int(temp[1])
    if gender == 0:                                # Move to Male folder
        new_path = os.path.join(m_dir, filename)
        shutil.move(image_path, new_path)
    elif gender == 1:                              # Move to Female folder
        new_path = os.path.join(f_dir, filename)
        shutil.move(image_path, new_path)

In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(TEST_SORT_DIR, 
                                                      seed=123, 
                                                      batch_size=128)

Found 23708 files belonging to 2 classes.


In [ ]:
# Prediction function

def run_predict()
# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print(classification_report(train_labels,train_predictions_rounded_labels)) 

## VGG-16

### Model 1b-gs: Grayscale + Frozen layers

In [ ]:
# Recreate model instance


# Restore the weights
model.load_weights('GC weights\Frozen\mod1b-gs\mod1b_gs_frozen.h5')



